#### LeetCode - SQL - #1045

Refer [1045. Customers Who Bought All Products](https://leetcode.com/problems/customers-who-bought-all-products/description/)

Write a solution to report the customer ids from the  `Customer`  table that bought all the products in the  `Product`  table.

Return the result table in  **any order**.

In [0]:
data_customer = [[1, 5], [2, 6], [3, 5], [3, 6], [1, 6]]
columns_customer = ['customer_id', 'product_key']
schema_customer = {'customer_id':'Int64', 'product_key':'Int64'}

data_product = [[5], [6]]
columns_product = ['product_key']
schema_product = {'product_key':'Int64'}

In [0]:
from pyspark.sql import SparkSession

spark_context = SparkSession.builder.appName('LeetCode SQL').getOrCreate()

In [0]:
# Create Spark Dataframe using Pandas
import pandas

pandas_df_customer = pandas.DataFrame(data=data_customer, columns=columns_customer).astype(schema_customer)
pandas_df_product = pandas.DataFrame(data=data_product, columns=columns_product).astype(schema_product)

df_customer = spark_context.createDataFrame(pandas_df_customer)
df_product = spark_context.createDataFrame(pandas_df_product)

In [0]:
# Create Spark Dataframe using PySpark functions - 1
schema_customer = 'customer_id integer, product_key integer'
schema_product = 'product_key integer'

df_customer = spark_context.createDataFrame(data=data_customer, schema=schema_customer)
df_product = spark_context.createDataFrame(data=data_product, schema=schema_product)

In [0]:
# Create Spark Dataframe using PySpark functions - 2
from pyspark.sql.types import StructType, StructField, IntegerType

schema_customer = StructType([
    StructField('customer_id', IntegerType(), False),
    StructField('product_key', IntegerType(), False)
])
df_customer = spark_context.createDataFrame(data=data_customer, schema=schema_customer)

schema_product = StructType([
    StructField('product_key', IntegerType(), False)
])
df_product = spark_context.createDataFrame(data=data_product, schema=schema_product)

In [0]:
from pyspark.sql import functions as F

df_product_count = df_product.select(F.count(df_product.product_key))
product_count = df_product_count.first()[0]

In [0]:
df_customer_grouped \
    = df_customer \
        .groupBy(df_customer.customer_id) \
        .agg(
            F.countDistinct(df_customer.product_key).alias('count_distinct_product')
        )

df_customer_grouped \
    .filter(df_customer_grouped.count_distinct_product == product_count) \
    .select(df_customer_grouped.customer_id) \
    .show()
    

+-----------+
|customer_id|
+-----------+
|          1|
|          3|
+-----------+

